# TP3ex1



Com base no artigo de <i> Joseph Silverman </i> implementa-mos o esquema de assinaturas digitais NTRU Sign.




In [40]:
from sage.crypto.util import ascii_to_bin

Numa primeira fase na implementação do NTRU Sign começamos por gerar os parametros necessários e de seguida gerarmos os anéis necessários.

In [41]:
#parametros do NTRU Silverman

N = 661
p = 3
q = 9829081
B = 1487
k = 10

In [42]:
#Gerar os aneis (página 20)

Zx.<x> = PolynomialRing(ZZ)
Qx.<x> = PolynomialRing(GF(q))
Px.<x> = PolynomialRing(GF(p))

quo = x^N + 1

Zxr.<x> = Zx.quotient(quo)
Qxr.<x> = Qx.quotient(quo)
Pxr.<x> = Px.quotient(quo)

In [43]:

class NTRUSign:
    def __init__(self):
        #(página 30)
        
        # Parametros da chave pública e assinatura da chave
    
        # Gerar f e F
        while True:
            F=Zxr(Pxr.random_element())      #f random mod 3
            f =  F * p                       #f=pF
            if Qxr(f).is_unit() :
                break
                
        # Gerar g
        while True:
            g = Zxr(Pxr.random_element())     #g random mod p
            if Pxr(g).is_unit():
                break
           
        # o par de vetores (f,g) representa a chave da assinatura
        self.priv_key = (f, g)

        
        
        # Processo de verificação da chave gerada anteriormente
        #chave publica  =>   h= f^-1* g (mod q)
        
        self.h= 1 / Qxr(f) * Qxr(g)
        

  
     #---- Digital Documents ---
    def dochash(self, m):
        #Teria de se verificar que (s,t) é próximo de (m1,m2). 
        #Desse modo, dividiu-se a mensagem em duas partes de igual tamanho.
        
        #lista com metade do tamanho da mensagem
        size = floor(len(m)/2)
        #vetor 1  tem metade para a frente da mensagem
        one = m[:size]
        #vetor 2  primeira metade
        two = m[size:]
        res1 = list()
        res2 = list()

 
        #Converteu-se cada elemento de caracter para número (ord).
        #Depois colocou-se esse valor em mod 2, assim só retorna 0 e 1. 
        #Para sair do ciclo, tem de se subtrair 2 e o próprio valor do elemento, ficando um valor negativo.
        
        #primeiro vetor
        for ele in one:
            ascii = ord(ele)
            while ascii >= 0:
                rest = ascii % 2
                res1.append(rest)
                ascii = ascii - 2 - rest

        # segundo vetor
        for ele in two:
            ascii = ord(ele)
            while ascii >= 0:
                rest = ascii %2
                res2.append(rest)
                ascii = ascii - 2 - rest 
                
                
                h1=Zx(res1)
                h2=Zx(res2)
        return (h1,h2)

    # assinatura da mensagem
    def assinar(self,(sp, tp)):
        #(pagina 31)
        boundary = floor(q/2*p - 0.5)       #o valor de r tem de ser menor ou igual a q/2*p - 0.5 
        while True:
            while True:
                r = Zxr.random_element()
                if abs(min(r)) <= boundary:
                    break
            s0 = Zxr(sp) + p*r                                        #s0=sp+pr
            t0 = Qxr(self.h) * Qxr(s0)     #t0=h*s0 (mod q)
            a = 1/Pxr(self.priv_key[1]) * (Pxr(tp) - Pxr(Zxr(t0)))  #a=g^-1*(tp-t0)(mod p)
            s = Zxr(s0) + Zxr(a) * Zxr(self.priv_key[0])     #s=s0+a*f #f estÃ¡ na posiÃ§Ã£o 0 dos vetores
            t = Zxr(t0) + Zxr(a) * Zxr(self.priv_key[1])     #t=t0+a*g #g estÃ¡ na posiÃ§Ã£o 1 dos vetores
            if abs(min(s)) <= floor(q*0.5 - B) or abs(min(t)) <= floor(q*0.5 - B) :     #confirma-se que q Ã© menor que o limite estipulado
                break
        return (s,t)

    def validar(self, (sp, tp), (s,t)):
        boundary = floor(q*0.5 - B)
        if Qxr(s) * Qxr(self.h) == Qxr(t) and (Pxr(s), Pxr(t)) == (Pxr(sp), Pxr(tp)) and abs(min(s)) and abs(min(t)) <= boundary:
            return True
        return False




In [44]:
K = NTRUSign()
m = "Mensagem de para assinar."
(sp, tp) = K.dochash(m)
(s, t) = K.assinar((sp, tp))
print(K.validar((sp, tp), (s, t)))

True
